In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

In [4]:
import gensim
from src.models.Word2VecModel import Word2VecModel
from src.data.WordData import WordData
import json
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

from keras.models import load_model
from sklearn.cluster import KMeans, DBSCAN

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
OUTPUT_DIM = 100
WINDOWS_SIZE = 2

In [6]:
with open('{}/lemma.json'.format('../data/interim')) as f:
    tokens = json.load(f)
    
word_data = WordData(tokens, None, 0, False, False)

In [7]:
# w2v = gensim.models.KeyedVectors.load_word2vec_format('../models/floydhub_w2v/vectors.txt', binary=False)
w2v = gensim.models.KeyedVectors.load_word2vec_format('../models/word2vec/vectors.txt', binary=False)

In [8]:
sent2vec = np.zeros((len(word_data.token_final), OUTPUT_DIM))
for i, sent in enumerate(word_data.token_final):
    sent2vec[i, : ] = np.mean(w2v.vectors[[ word_data.ref_word_to_id[token] for token in sent ]], axis = 0)

In [9]:
sent2vec.shape

(36219, 100)

In [39]:
dbscan = DBSCAN(eps = 0.2, metric = 'cosine', min_samples = 2)

## DBSCAN par plage de date

### calculs des dates

In [11]:
dates = pd.read_csv('../data/external/csv_datas_full.csv', sep = '\t', low_memory=False)

In [12]:
dates['tweet_creation_dt'] = pd.to_datetime(dates['tweet_creation_dt'])
dates['tweet_creation_dt'] = dates['tweet_creation_dt'].dt.date

In [13]:
count = dates.groupby('tweet_creation_dt')['tweet_creation_dt'].count().reset_index(name = 'count')
count

,tweet_creation_dt,count
0,2015-01-02,1
1,2015-01-03,1
2,2015-01-04,15
3,2015-01-05,2
4,2015-01-06,10
5,2015-01-07,6
6,2015-01-08,18
7,2015-01-09,6
8,2015-01-10,11
9,2015-01-11,9


In [14]:
count.describe()

,count
count,1296.000000
mean,27.946759
std,53.013594
min,1.000000
25%,7.000000
50%,14.500000
75%,29.000000
max,508.000000


In [15]:
dates.sort_values(by = 'tweet_creation_dt', inplace=True)

In [16]:
dates = dates['tweet_creation_dt']

In [17]:
dates.head()

36210    2015-01-02
29299    2015-01-03
17212    2015-01-04
14685    2015-01-04
17213    2015-01-04
Name: tweet_creation_dt, dtype: object

In [18]:
def getNonSimilarToken(tokens):
    dbscan.fit(tokens)
    labels = dbscan.labels_
    return labels

In [40]:

WINDOWS_SIZE = 30
STEP_SIZE = 15
MIN_DATE = min(dates)
MAX_DATE = max(dates)

borne_inf = MIN_DATE
borne_sup = borne_inf + timedelta(days = WINDOWS_SIZE)

X = 0
token_id_keep = []

while borne_inf <= MAX_DATE:
    print("NOUVELLE PLAGE")
    ## selection sur les lignes
    tweets_sel = dates[dates >= borne_inf]
    tweets_sel = tweets_sel[tweets_sel < borne_sup]
    
    ## fonction
    labels = getNonSimilarToken(sent2vec[tweets_sel.index])
    X += np.sum(labels == -1)
    
    labels_cluster = []
    
    for i, t in enumerate(tweets_sel.index):
        if labels[i] == -1:
            token_id_keep.append(t)
        elif labels[i] not in labels_cluster:
            token_id_keep.append(t)
            labels_cluster.append(labels[i])
        if labels[i]==2:
            print(str(t)+' '+' '.join(word_data.token_final[t]))
    
    ## calcul des bornes
    borne_inf = borne_inf + timedelta(days = STEP_SIZE)
    borne_sup = borne_inf + timedelta(days = WINDOWS_SIZE)

print(X)

NOUVELLE PLAGE
14686 pour le jo de 2024 le normandie se positionner pour le epreuves de voiler jo2024
14683 pour le jo2024 le normandie se positionner pour le epreuves de voiler via normandieactu
NOUVELLE PLAGE
14666 jo2024 le normandie poser son candidature pour le epreuves de voile de paris paris2024 normandie2024
14664 rt sportsociete jo2024 le normandie poser son candidature pour le epreuves de voile de paris paris2024
NOUVELLE PLAGE
14666 jo2024 le normandie poser son candidature pour le epreuves de voile de paris paris2024 normandie2024
14664 rt sportsociete jo2024 le normandie poser son candidature pour le epreuves de voile de paris paris2024
NOUVELLE PLAGE
35992 lcpan paris lorgner sur le jo2024 sport athletes
35994 esjparis paris lorgner sur le jo2024 sport athletes
35995 le_parisien paris lorgner sur le jo2024 sport athletes
NOUVELLE PLAGE
13314 le match pour le extension de roland garros se jouer au tie-break rg2015 urbanisme jo2024
17678 le match pour le extension de roland

1424 trop cher le jeuxolympiques plaire personne ne en vouloir avoir partir le politicien pour le prestige personnel paris2024
1439 trop cher le jeuxolympiques plaire personne ne en vouloir avoir partir le politicien pour le prestige personnel paris2024
NOUVELLE PLAGE
4699 le mouvement sportif de le ssd93 se mobiliser pour le candidature de paris2024
1308 rio2016 paris2024 le photo de famille de le delegation officiel de le candidature de pari pour le jo de 2024
13486 le departement soutien le candidature de le ville de pari pour le jo en 2024 germinalpeiro paris2024
NOUVELLE PLAGE
4638 en direct sur periscope le bleu en router pour rio2016 j100rio espritbleu paris2024 roadtorio athletespar
4637 en direct sur periscope le bleu en router pour rio2016 j100rio espritbleu paris2024 roadtorio athletespar
NOUVELLE PLAGE
30804 demain anne_hidalgo presente le plan paris2024 illustration de un projet qui faire changer le societe par le sport
30801 demain anne_hidalgo presente le plan paris2024 

11869 venezpartager le 1er episode de le mini-serie objectif zero dechet paris2024 et suez engager pour un jeu circulaire et durable
11846 venezpartager le 2e episode de le mini-serie objectif zerodechet paris2024 et suezfr engager pour un jeu circulaire et durable
11825 rt paris2024 venezpartager le 2e episode de le mini-serie objectif zerodechet paris2024 et suezfr engager pour un jeu
11774 rt paris2024 venezpartager le 3e episode de le mini-serie objectif zerodechet paris2024 et suezfr engager pour un jeu
11783 venezpartager le 3e episode de le mini-serie objectif zerodechet paris2024 et suezfr engager pour un jeu circulaire et durable
NOUVELLE PLAGE
9816 avoir j-100 de le decision final pour le organisation un jo2024 tonyestanguet inviter de rfi paris2024 #venezpartager fredriviererfi
9875 j-100 avant le decision final pour le organisation un jo2024 tonyestanguet être le inviter de rfi ce matin paris2024 #venezpartager
NOUVELLE PLAGE
10381 macron paris2024 policybrief par gerauntr 

3549 avoir le heure un jeuxolympiques de pyeongchang2018 et pour patienter avant le jo de paris2024 letsgofrance revenir sur le jo de pari 1924 > > paris2024
3543 avoir le heure un jeuxolympiques de pyeongchang2018 et pour patienter avant le jo de paris2024 letsgofrance revenir sur le jo de pari 1924 > >
56461


In [41]:
token_id_keep = list(set(token_id_keep))

In [42]:
len(token_id_keep)

30735

In [73]:
with open('../data/interim/tokens_id_keep_2.json', 'w') as outfile:
    json.dump(token_id_keep, outfile)

In [33]:
with open('../data/interim/tokens_id_keep.json') as f:
    tmp = json.load(f)



In [34]:
len(tmp)

30735